In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date, timedelta
import konlpy as kn
import MeCab as mc
import ekonlpy as ek
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [2]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-02-01 00:00:00
2024-01-30


In [3]:
f = open('./crawl_result/call_rate.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['date', 'price', 'label']
['2024-03-21\t3.5290\t-0.11\n', '2024-03-20\t3.5330\t0.56\n', '2024-03-19\t3.5130\t2.09\n', '2024-03-18\t3.4410\t-0.66\n', '2024-03-15\t3.4640\t-0.63\n']


In [4]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None,
    min_difference = pd.Timedelta.max

    for date in df['date']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def label_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['date', 'price', 'label'])

    # 'date' 열을 datetime 형식으로 변환
    df['date'] = pd.to_datetime(df['date'])

    for i, row in df.iterrows():
        time = row['date']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인
        if time2 in df['date'].values:
            prev_row = df[df['date'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['date'] == nearest_date].iloc[0]

        if row['price'] > prev_row['price']:
            label = '1'
        elif row['price'] == prev_row['price']:
            label = 0
        else:
            label = '-1'
            
        df_ud.loc[i] = [time.date(), row['price'], label]

    return df_ud 

In [5]:
df = pd.DataFrame(new_datas)

call = label_call(df)
call

,date,price,label
0,2024-03-21,3.5290,-1
1,2024-03-20,3.5330,-1
2,2024-03-19,3.5130,-1
3,2024-03-18,3.4410,-1
4,2024-03-15,3.4640,-1
...,...,...,...
3253,2011-01-07,2.4900,0
3254,2011-01-06,2.4900,0
3255,2011-01-05,2.4900,0
3256,2011-01-04,2.5000,1


In [6]:
# 문장 분할을 위한 정규표현식 패턴
sentence_pattern = r'(?<=[.!?]) +'

# 데이터프레임으로부터 문서를 읽어와 딕셔너리 구조로 저장하는 함수
def read_documents_to_dict(df):
    date_dict = {}
    change_pattern = '(?<=[가-힣])\\.'
    news_pattern = ['\\[.*\\]', '\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?', '....기자']

    # DataFrame의 각 행에 대해 반복
    for index, row in df.iterrows():
        date = str(row['date']).split(' ')[0] 
        text = row['content'] 
        
        # 뉴스 패턴 제거
        news_text = text
        for pattern in news_pattern:
            news_text = re.sub(pattern, '', news_text)

        # 변경 패턴 적용
        change_text = re.sub(change_pattern, '@@@', news_text)  
        split_text = change_text.split('@@@')  
        split_text = [sent.strip() for sent in split_text]
        
        # 문서별로 문장 추가
        if date not in date_dict:
            date_dict[date] = []
        
        # 문장 추가
        if split_text:  # 빈 리스트인 경우 추가하지 않음
            date_dict[date].append(split_text)

    return date_dict

# 문서 데이터프레임 읽기
# -------------------------------------------------------------------------------------예시 경로 설정 필요 (여기만 건드시면 됩니다 함수 안건드셔도되요)-------------------------------------
jong_path = "./crawl_result/joongamg_news_test.json"
edaily_path = "./crawl_result/edaily_news_test.json"
money_path = "./crawl_result/moneytoday_news_test.json"
bone_path = "./crawl_result/bone_report.json"
min_path = "./crawl_result/minutes_re.tsv"

df_news_1 = pd.read_json(jong_path)
df_news_2 = pd.read_json(edaily_path)
df_news_3 = pd.read_json(money_path)
df_bone = pd.read_json(bone_path)
df_min = pd.read_csv(min_path, sep='\t')

# 문서들을 딕셔너리 구조로 저장
date_dict_1 = read_documents_to_dict(df_news_1)
date_dict_2 = read_documents_to_dict(df_news_2)
date_dict_3 = read_documents_to_dict(df_news_3)
date_dict_4 = read_documents_to_dict(df_bone)
date_dict_5 = read_documents_to_dict(df_min)

In [7]:
# 두 개의 딕셔너리를 합치는 함수
def merge_dicts(dict1, dict2):
    merged_dict = dict1.copy()
    for key, value in dict2.items():
        if key in merged_dict:
            merged_dict[key].extend(value)  # 이미 있는 키의 경우 값들을 합침
        else:
            merged_dict[key] = value  # 새로운 키의 경우 값을 그대로 추가
    return merged_dict



# 두 개의 딕셔너리 합치기 -------------------------- 함수 건들지 마시고 밑에꺼에 dict 값 넣으시면 됩니다. ------------------------------------------------
merged_date_dict_1 = merge_dicts(date_dict_1, date_dict_2)
merged_date_dict_2 = merge_dicts(date_dict_3, date_dict_4)
merged_date_dict_3 = merge_dicts(merged_date_dict_1, merged_date_dict_2)

final_dict = merge_dicts(merged_date_dict_3, date_dict_5)

In [8]:
final_dict['2013-03-14'][0]
####  딕셔너리 이름 [날짜] [몇번째 문서인지] [그 문서의 몇번째 문장인지]

['스페인의 국채 낙찰금리가 또다시 하락하는 등 입찰이 호조를 이어가고 있다',
 '이탈리아 정국 혼란 속에서도 투자자들의 신뢰가 높아진 덕으로 풀이된다',
 '스페인 재무부는 14일(현지시간) 입찰을 통해 8억300유로(10억4000만달러) 어치의 국채를 발행하는데 성공했다',
 '만기가 오는 2029년인 15년물 국채의 낙찰금리는 5.224%로, 지난달 입찰에서의 5.787%에 더 낮아졌다',
 '또 만기가 2040년인 장기국채 역시 낙찰금리가 5.434%로, 지난해 12월 입찰에서의 5.893%보다 내려갔다',
 '만기 2041년 국채 역시 5.432%로, 지난 1월의 5.686%보다 낮아졌다',
 '이번 입찰은 당초 예정에 없던 것으로, 이번주초 10년만기 국채금리가 지난 2010년 11월 이후 최저수준으로 내려가자 저금리에 자금 조달을 위해 임의로 편성한 것이다',
 '제스틴 나이트 UBS 금리담당 스트래티지스트는 “스페인 재무부가 특별 입찰을 실시할 정도로 시장 내에서 국채를 인수하려는 수요가 많다는 의미”라고 평가했다',
 '실제 이날 입찰에서 만기 15년물의 경우 입찰액대비 응찰비율이 4.1배에 이르러 한 달전의 2.02배에 비해 2배 이상 수요가 늘어났다',
 '']

In [16]:
from ekonlpy.tag import Mecab

# ekonlpy의 Mecab 형태소 분석기 인스턴스 생성
tokenizer = Mecab()

# n-gram 생성 함수 정의
def generate_ngrams(tokens, n):
    ngrams = []
    for i in range(len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))  # 각 n-gram을 공백으로 구분된 문자열로 결합
    return ngrams

# 토큰화 및 n-gram 생성 함수 정의
def tokenize_with_ngrams(sentence, n):
    tokens_with_pos = tokenizer.pos(sentence)
    tokens = ['{}/{}'.format(token, pos) for token, pos in tokens_with_pos if pos in ['VA', 'VX', 'VV', 'NNG', 'MAG']]
    ngrams = generate_ngrams(tokens, n)
    return [tuple(ngram.split()) for ngram in ngrams]

# 토큰화된 결과를 데이터프레임으로 출력
token_data = {'date': [], 'document': [], 'tokens': []}
n = 1  # n-gram의 n값 설정
count = 0  # 출력된 결과의 수를 세는 변수

for date, documents in final_dict.items():
    for doc_num, sentences in enumerate(documents, start=1):
        # print(f"{date}의 문서 {doc_num} 토큰화 및 {n}-gram 생성 결과:")
        for sentence in sentences:
            ngram_tokens = tokenize_with_ngrams(sentence, n)
            token_data['date'].append(date)
            token_data['document'].append(f"문서 {doc_num}")
            token_data['tokens'].append(ngram_tokens)
            count += 1
            if count >= 10:  # 처음 10개의 결과만 확인하고 루프 종료
                break
        else:
            continue
        break
    else:
        continue
    break

df = pd.DataFrame(token_data)
df = df[df['tokens'].apply(lambda x: len(x) > 0)]

# df.to_csv("1111.csv", mode='w')

In [39]:
# 'date' 열을 datetime으로 변환
call['date'] = pd.to_datetime(call['date'])
df['date'] = pd.to_datetime(df['date'])

# 'date' 열을 기준으로 외부 조인 수행
merged_df = pd.merge(call, df, on='date', how='outer')

# 'key' 칼럼을 인덱스로 설정
merged_df.set_index('date', inplace=True)

# print(merged_df)

In [40]:
# 'label' 칼럼의 값이 '1(매파)'인 행과 '-1(비둘기파)'인 행으로 데이터프레임 분리
df_P = merged_df[merged_df['label'] == '1']
df_N = merged_df[merged_df['label'] == '-1']

# Nan인 행 제거
df_P = df_P.dropna()
df_N = df_N.dropna()

# 필요없는 열 제거 
df_P = df_P.drop(['price'], axis=1) # 'date'
df_N = df_N.drop(['price'], axis=1) # 'date'

# word_df = pd.concat([df_P,df_N],axis=0)
# x_tain = word_df["words"]
# y_tain = word_df["label"]

In [41]:
df_P = df_P.rename(columns={'label': 'P'})
df_P
df_P.to_csv("df_P_5gram.csv", mode='w')

In [42]:
df_N = df_N.rename(columns={'label': 'N'})
df_N
df_N.to_csv("df_N_5gram.csv", mode='w')